In [1]:
from bs4 import BeautifulSoup # biblioteca para análisis y extracción de datos de archivos HTML y XML.
import requests
import pandas as pd

In [2]:
url = 'https://en.wikipedia.org/wiki/Life-cycle_greenhouse_gas_emissions_of_energy_sources'
res = requests.get(url)
print(res.status_code)

200


In [3]:
soup = BeautifulSoup(res.content, 'html.parser')

In [4]:
tablas = soup.find_all("table")
print(type(soup))
print("El número de tablas que tenemos en la página web es:", len(tablas))

<class 'bs4.BeautifulSoup'>
El número de tablas que tenemos en la página web es: 6


In [5]:
print("En la tabla 3 tenemos: \n ", tablas[3])

En la tabla 3 tenemos: 
  <table class="wikitable sortable">
<caption>Life cycle CO<sub>2</sub> equivalent (including <a href="/wiki/Albedo" title="Albedo">albedo</a> effect) from selected electricity supply technologies according to IPCC 2014.<sup class="reference" id="cite_ref-IPCC_2014_Annex_III_3-1"><a href="#cite_note-IPCC_2014_Annex_III-3">[3]</a></sup><sup class="reference" id="cite_ref-4"><a href="#cite_note-4">[4]</a></sup> Arranged by decreasing <a href="/wiki/Median" title="Median">median</a> (gCO<sub style="font-size: 80%;vertical-align: -0.35em">2</sub>eq/kWh) values.
</caption>
<tbody><tr>
<th>Technology</th>
<th>Min.</th>
<th>Median</th>
<th>Max.
</th></tr>
<tr>
<th colspan="4">Currently commercially available technologies
</th></tr>
<tr>
<td><a href="/wiki/Coal" title="Coal">Coal</a> – <a href="/wiki/Coal_burner" title="Coal burner">PC</a></td>
<td align="right">740</td>
<td align="right">820</td>
<td align="right">910
</td></tr>
<tr>
<td><a class="mw-redirect" href="/w

In [6]:
data = tablas[3]

dictionary = {'Technology': [],
              'Min.': [],
              'Median': [],
              'Max.': [],
              'Currently commercially available technologies': []
            }

In [7]:
print(type(data.findAll("tr")))

<class 'bs4.element.ResultSet'>


In [8]:
medalleros = tablas[3]

dictionary = {'Technology': [],
              'Min.': [],
              'Median': [],
              'Max.': [],
              'Currently commercially available technologies': [],
              'Pre‐commercial technologies': []
            }

In [9]:
print(type(medalleros.findAll("tr")))

<class 'bs4.element.ResultSet'>


In [10]:
lista_encabezados = medalleros.find_all("th")
lista_encabezados

[<th>Technology</th>,
 <th>Min.</th>,
 <th>Median</th>,
 <th>Max.
 </th>,
 <th colspan="4">Currently commercially available technologies
 </th>,
 <th colspan="4">Pre‐commercial technologies
 </th>]

In [11]:
encabezados = []
for columna in lista_encabezados:
    encabezados.append(columna.text)

encabezados

['Technology',
 'Min.',
 'Median',
 'Max.\n',
 'Currently commercially available technologies\n',
 'Pre‐commercial technologies\n']

In [12]:
encabezados_limpios = [element.replace('\n', '') for element in encabezados]
encabezados_limpios

['Technology',
 'Min.',
 'Median',
 'Max.',
 'Currently commercially available technologies',
 'Pre‐commercial technologies']

In [13]:
medalleros = medalleros.findAll("tr")[2:]
medalleros

[<tr>
 <td><a href="/wiki/Coal" title="Coal">Coal</a> – <a href="/wiki/Coal_burner" title="Coal burner">PC</a></td>
 <td align="right">740</td>
 <td align="right">820</td>
 <td align="right">910
 </td></tr>,
 <tr>
 <td><a class="mw-redirect" href="/wiki/Fossil_gas" title="Fossil gas">Gas</a> – <a class="mw-redirect" href="/wiki/Combined_cycle" title="Combined cycle">combined cycle</a></td>
 <td align="right">410</td>
 <td align="right">490</td>
 <td align="right">650
 </td></tr>,
 <tr>
 <td><a href="/wiki/Biomass" title="Biomass">Biomass</a> – Dedicated</td>
 <td align="right">130</td>
 <td align="right">230</td>
 <td align="right">420
 </td></tr>,
 <tr>
 <td><a class="mw-redirect" href="/wiki/Solar_PV" title="Solar PV">Solar PV</a> – Utility scale</td>
 <td align="right">18</td>
 <td align="right">48</td>
 <td align="right">180
 </td></tr>,
 <tr>
 <td><a class="mw-redirect" href="/wiki/Solar_PV" title="Solar PV">Solar PV</a> – rooftop</td>
 <td align="right">26</td>
 <td align="right"

In [23]:
dictionary = {"Technology": [], "Min.": [], "Median": [], "Max.": []}

for row in medalleros:
    cells = row.find_all("td")
    
    if len(cells) >= 4:
        dictionary["Technology"].append(cells[0].text.strip())
        dictionary["Min."].append(cells[1].text.strip())
        dictionary["Median"].append(cells[2].text.strip())
        dictionary["Max."].append(cells[3].text.strip())
    else:
        print("Skipping row - not enough cells")

print(dictionary)

Skipping row - not enough cells
{'Technology': ['Coal – PC', 'Gas – combined cycle', 'Biomass – Dedicated', 'Solar PV – Utility scale', 'Solar PV – rooftop', 'Geothermal', 'Concentrated solar power', 'Hydropower', 'Wind Offshore', 'Nuclear', 'Wind Onshore', 'Ocean (Tidal and wave)'], 'Min.': ['740', '410', '130', '18', '26', '6.0', '8.8', '1.0', '8.0', '3.7', '7.0', '5.6'], 'Median': ['820', '490', '230', '48', '41', '38', '27', '24', '12', '12', '11', '17'], 'Max.': ['910', '650', '420', '180', '60', '79', '63', '22001', '35', '110', '56', '28']}


In [15]:
df = pd.DataFrame(dictionary)
df

,Technology,Min.,Median,Max.
0,Coal – PC,740,820,910
1,Gas – combined cycle,410,490,650
2,Biomass – Dedicated,130,230,420
3,Solar PV – Utility scale,18,48,180
4,Solar PV – rooftop,26,41,60
5,Geothermal,6.0,38,79
6,Concentrated solar power,8.8,27,63
7,Hydropower,1.0,24,22001
8,Wind Offshore,8.0,12,35
9,Nuclear,3.7,12,110


In [18]:
# Save DataFrame to a CSV file
df.to_csv('lifecicleWIKI.csv', index=False)

In [ ]:
df = pd.DataFrame(dictionary)


In [ ]:
df

In [ ]:
CHANGES NAMES FROM MEDALLEROS AND MEDALLERO CHECK IT OUT